# Get Pretrained Models
Okay

In [3]:
!mkdir -p checkpoints
!wget https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_b.pt -P checkpoints
!wget https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_blt.pt -P checkpoints
!wget https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_s0.pt -P checkpoints
!wget https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_s1.pt -P checkpoints
!wget https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_s2.pt -P checkpoints

--2025-04-30 17:44:39--  https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_b.pt
Resolving docs-assets.developer.apple.com (docs-assets.developer.apple.com)... 17.253.73.202, 17.253.73.201
Connecting to docs-assets.developer.apple.com (docs-assets.developer.apple.com)|17.253.73.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 599218731 (571M) [application/octet-stream]
Saving to: ‘checkpoints/mobileclip_b.pt’

mobileclip_b.pt      78%[==============>     ] 447,99M  5,24MB/s    eta 22s    

OSError: [Errno 5] Input/output error

In [ ]:
import torch
from PIL import Image
import mobileclip

model, _, preprocess = mobileclip.create_model_and_transforms('mobileclip_s0', pretrained='/path/to/mobileclip_s0.pt')
tokenizer = mobileclip.get_tokenizer('mobileclip_s0')

image = preprocess(Image.open("docs/fig_accuracy_latency.png").convert('RGB')).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a cat"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)